In [18]:
from kafka import KafkaConsumer
from datetime import datetime
from s3fs import S3FileSystem
from json import loads, dump
import configparser

In [8]:
config = configparser.ConfigParser()
config.read('config.ini')

# Read credentials
ip_address = config.get('AWS','ip_address')
topic_name = config.get('KAFKA','topic_name')
port = config.get('KAFKA','port')
bucket_name = config.get('AWS','bucket_name')

In [7]:
consumer = KafkaConsumer(
                        topic_name,
                        bootstrap_servers=f'{ip_address}:{port}',
                        #auto_offset_reset='earliest',
                        #enable_auto_commit=True,
                        #group_id='my-group',
                        value_deserializer=lambda x: loads(x.decode('utf-8')))

s3 = S3FileSystem()

In [22]:
for message in consumer:
    message =  message.value
    
    for stock in message['quoteResponse']['result']:
        dictionary = {}
        if 'longName' in stock.keys():
                dictionary['name'] = stock['longName']
                dictionary['symbol'] = stock['symbol']
                dictionary['currency'] = stock['currency']
                #dictionary['market_time'] = datetime.fromtimestamp(stock['regularMarketTime'])
                dictionary['market_time'] = stock['regularMarketTime']
                dictionary['actual_price'] = stock['regularMarketPrice']
                dictionary['previous_close'] = stock['regularMarketPreviousClose']
                dictionary['market_open'] = stock['regularMarketOpen']
                dictionary['low'] = stock['regularMarketDayLow']
                dictionary['high'] = stock['regularMarketDayHigh']
                dictionary['volume'] = stock['regularMarketVolume']

                document_name = dictionary['symbol']

                with s3.open(f's3://{bucket_name}/stock_market_{document_name}.json', 'w') as f:
                    dump(dictionary, f)

        else: print('Unknown stock: ', stock['symbol'])


KeyboardInterrupt: 